In [1]:
# !pip install transformers datasets peft accelerate bitsandbytes trl
# !pip install -U transformers datasets peft accelerate bitsandbytes trl reportlab


In [2]:
import transformers
print(transformers.__version__)  # should be >= 4.54.0

from transformers import TrainingArguments
print(TrainingArguments.__init__.__code__.co_varnames)


4.53.3
('self', 'output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size', 'gradient_accumulation_steps', 'eval_accumulation_steps', 'eval_delay', 'torch_empty_cache_steps', 'learning_rate', 'weight_decay', 'adam_beta1', 'adam_beta2', 'adam_epsilon', 'max_grad_norm', 'num_train_epochs', 'max_steps', 'lr_scheduler_type', 'lr_scheduler_kwargs', 'warmup_ratio', 'warmup_steps', 'log_level', 'log_level_replica', 'log_on_each_node', 'logging_dir', 'logging_strategy', 'logging_first_step', 'logging_steps', 'logging_nan_inf_filter', 'save_strategy', 'save_steps', 'save_total_limit', 'save_safetensors', 'save_on_each_node', 'save_only_model', 'restore_callback_states_from_checkpoint', 'no_cuda', 'use_cpu', 'use_mps_device', 'seed', 'data_seed', 'jit_mode_eval', 'use_ipex', 'bf16', 'fp16', 'fp16_opt_level', 'half_precision_ba

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [4]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Load the dataset from Hugging Face
dataset = load_dataset("tatsu-lab/alpaca")
'''
data = {
    "question": [
        "What is artificial intelligence?",
        "Who wrote the book '1984'?",
        "What is the capital of France?"
    ],
    "answer": [
        "Artificial intelligence is the simulation of human intelligence in machines.",
        "George Orwell wrote the book '1984'.",
        "The capital of France is Paris."
    ]
}
'''
print(dataset)
print(dataset["train"][0])  # Show first example
'''
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 10570
    })
})

'''



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


"\nDatasetDict({\n    train: Dataset({\n        features: ['instruction', 'input', 'output', 'text'],\n        num_rows: 87599\n    })\n    validation: Dataset({\n        features: ['instruction', 'input', 'output', 'text'],\n        num_rows: 10570\n    })\n})\n\n"

In [5]:
# Cell 1
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq

MODEL_NAME = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
model_name = MODEL_NAME  # For later use (PDF report)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})

In [7]:
# Cell 2
from datasets import load_dataset

# Load Alpaca dataset
dataset = load_dataset("yahma/alpaca-cleaned")

# Create train/test split if not already present
if "test" not in dataset:
    dataset = dataset["train"].train_test_split(test_size=0.3)


README.md: 0.00B [00:00, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 36232
    })
    test: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 15528
    })
})

In [9]:
# Cell 3
prefix = "Please answer this question: "

def preprocess_function(examples):
    """Add prefix, tokenize input and output."""
    if "input" in examples:
        inputs = [
            prefix + instr + (" " + inp if inp else "")
            for instr, inp in zip(examples["instruction"], examples["input"])
        ]
    else:
        inputs = [prefix + instr for instr in examples["instruction"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    labels = tokenizer(
        text_target=examples["output"],
        max_length=512,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [10]:
print(dataset.column_names)


{'train': ['output', 'input', 'instruction'], 'test': ['output', 'input', 'instruction']}


In [11]:
# Cell 4
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/36232 [00:00<?, ? examples/s]

Map:   0%|          | 0/15528 [00:00<?, ? examples/s]

In [12]:
# Cell 5
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


In [13]:
!pip install reportlab
!pip install evaluate rouge_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=aaf307d23a8a9a4e76094d02bd1afda4600bdda195d30cfe1dbf104c7b3b2862
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [14]:
# !pip install -U transformers


In [15]:
import numpy as np
import evaluate

rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1) if predictions.ndim > 1 else predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge_metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    return {key: value * 100 for key, value in result.items()}


In [16]:
import transformers
print(transformers.__version__)


4.53.3


In [21]:
# Cell 6 (Optimized for T4 GPU)
!pip install -q transformers datasets peft accelerate bitsandbytes trl reportlab

import os
import torch
import importlib
import transformers
importlib.reload(transformers)
from transformers import Trainer, TrainingArguments, set_seed
from datetime import datetime
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
import numpy as np
import evaluate

set_seed(42)

# Enable CUDA if available
if torch.cuda.is_available():
    model = model.to("cuda")
    torch.backends.cuda.matmul.allow_tf32 = True
    model.config.use_cache = False  # Disable cache for gradient checkpointing

# Ensure LoRA layers require gradients
for param in model.parameters():
    param.requires_grad = True


# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# ------------------------
# Metrics (ROUGE)
# ------------------------
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {key: round(value * 100, 2) for key, value in result.items()}

# ------------------------
# Training arguments
# ------------------------
output_dir = "./flan_t5_lora_alpaca"
batch_size = 2           # Reduced for T4 GPU
num_epochs = 3

args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=8,  # Higher accumulation to simulate bigger batch
    learning_rate=1e-4,
    num_train_epochs=num_epochs,
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    fp16=True,            # Enable mixed precision
    report_to="none"
)

# ------------------------
# Trainer
# ------------------------
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized_dataset["test"].select(range(min(200, len(tokenized_dataset["test"])))),
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-21-1535129341.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
# Cell 7
# Resume training if a checkpoint is
# Cell 7
from transformers.trainer_utils import get_last_checkpoint

last_checkpoint = get_last_checkpoint(output_dir)
if last_checkpoint is not None:
    print(f"Resuming from checkpoint: {last_checkpoint}")
    train_result = trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("No checkpoint found. Starting training from scratch.")
    train_result = trainer.train()

trainer.save_model(output_dir)


No checkpoint found. Starting training from scratch.


Step,Training Loss,Validation Loss


In [23]:
# Cell 8
def generate(model, tokenizer, instruction, input_text=None, max_new_tokens=128):
    if input_text and len(input_text.strip()) > 0:
        prompt = f"{prefix}{instruction}\n\nInput: {input_text}"
    else:
        prompt = f"{prefix}{instruction}"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=4
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_instructions = [
    ("Give three tips for staying healthy.", None),
    ("Explain overfitting in simple terms.", None),
    ("Translate to French:", "How are you today?"),
    ("Summarize:", "Large language models are trained on vast amounts of data to predict the next token."),
    ("Write a Python function to compute Fibonacci numbers.", None),
]

model.eval()
samples = []
for instr, inp in test_instructions:
    out = generate(model, tokenizer, instr, inp)
    samples.append((instr, inp, out))
    print("Instruction:", instr)
    if inp:
        print("Input:", inp)
    print("Model output:", out)
    print("=" * 80)


Instruction: Give three tips for staying healthy.
Model output: The three tips for staying healthy are: 1. Eat a healthy diet. 2. Eat a balanced diet. 3. Eat a balanced diet. 4. Eat a balanced diet. 5. Eat a balanced diet. 6. Eat a balanced diet. 7. Eat a balanced diet. 8. Eat a balanced diet. 9. Eat a balanced diet. 10. Eat a balanced diet. 11. Eat a balanced diet. 12. Eat a balanced diet. 13. Eat a balanced diet. 14. Eat a balanced diet. 15. Eat a balanced diet. 16. Eat a balanced diet. 17. Eat a balanced diet. 18.
Instruction: Explain overfitting in simple terms.
Model output: Overfitting is a term that refers to a number of factors that can cause a person to overfit. Examples of overfitting include: Overfitting is a term that refers to a number of factors that can cause a person to overfit. Examples of overfitting include: Overfitting is a term that refers to a number of factors that can cause a person to overfit. Examples of overfitting include: Overfitting is a term that refers t

In [24]:
# Cell 9
pdf_path = os.path.join(output_dir, "report.pdf")

def create_pdf(
    pdf_path,
    model_name,
    dataset_name,
    lora_cfg: LoraConfig,
    epochs,
    train_samples,
    eval_samples,
    sample_generations
):
    c = canvas.Canvas(pdf_path, pagesize=A4)
    width, height = A4
    margin = 40
    y = height - margin

    def write_line(text, size=10, lead=14):
        nonlocal y
        if y < margin + 50:
            c.showPage()
            y = height - margin
        c.setFont("Helvetica", size)
        c.drawString(margin, y, text)
        y -= lead

    write_line("Instruction Tuning Report", size=16, lead=20)
    write_line(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    write_line("")
    write_line("=== Model & Dataset ===", size=12, lead=16)
    write_line(f"Base model: {model_name}")
    write_line(f"Dataset: {dataset_name}")
    write_line("")
    write_line("=== LoRA Config ===", size=12, lead=16)
    write_line(f"r={lora_cfg.r}, alpha={lora_cfg.lora_alpha}, dropout={lora_cfg.lora_dropout}, bias={lora_cfg.bias}")
    write_line(f"task_type={lora_cfg.task_type}")
    write_line("")
    write_line("=== Training ===", size=12, lead=16)
    write_line(f"Epochs: {epochs}")
    write_line(f"Train samples (used): {train_samples}")
    write_line(f"Eval samples (used): {eval_samples}")
    write_line("")
    write_line("=== Sample Generations ===", size=12, lead=16)

    for i, (instr, inp, out) in enumerate(sample_generations, 1):
        write_line(f"{i}. Instruction: {instr}")
        if inp:
            write_line(f"   Input: {inp}")
        write_line(f"   Output: {out[:500]}")
        write_line("")

    c.save()

create_pdf(
    pdf_path=pdf_path,
    model_name=model_name,
    dataset_name="yahma/alpaca-cleaned",
    lora_cfg=lora_config,
    epochs=num_epochs,
    train_samples=2000,
    eval_samples=min(200, len(tokenized_dataset["test"])),
    sample_generations=samples
)

print(f"\nPDF report saved to: {pdf_path}")



PDF report saved to: ./flan_t5_lora_alpaca/report.pdf
